# Background

If we have several homologous strands that we wish to analyze simultaneously, then the natural problem is to find an average-case strand to represent the most likely common ancestor of the given strands. Say that we have a collection of DNA strings, all having the same length **n**. Their profile matrix is a 4×**n** matrix **P** in which P( 1, j ) represents the number of times that 'A' occurs in the jth position of one of the strings, P( 2, j ) represents the number of times that 'C' occurs in the jth position, and so on.

A consensus string c is a string of length n formed from our collection by taking the most common symbol at each position; the jth symbol of c therefore corresponds to the symbol having the maximum value in the j-th column of the profile matrix. Of course, there may be more than one most common symbol, leading to multiple possible consensus strings.



# Problem

Given: A collection of at most 10 DNA strings of equal length (at most 1 kbp) in FASTA format.

Return: A consensus string and profile matrix for the collection. (If several possible consensus strings exist, then you may return any one of them.)

## Sample Data
\>Rosalind_1 <br>
ATCCAGCT <br>
\>Rosalind_2<br>
GGGCAACT <br>
\>Rosalind_3<br>
ATGGATCT <br>
\>Rosalind_4<br>
AAGCAACC <br>
\>Rosalind_5<br>
TTGGAACT <br>
\>Rosalind_6<br>
ATGCCATT <br>
\>Rosalind_7<br>
ATGGCACT

## Sample Output

ATGCAACT <br>
A: 5 1 0 0 5 5 0 0 <br>
C: 0 0 1 4 2 0 6 1 <br>
G: 1 1 6 3 0 1 0 0 <br>
T: 1 5 0 0 0 1 1 6

# Solution

We have a number of DNA sequences with the same length and must keep count of how many times a nucleotide appears at each position among all sequences. Regular expressions can be used to find the index of each nucleotide along a given seq. A dictionary is usded to keep track of the nucleotides appearing at each position where the key is the nucleotide (A,C,G,T) and the value is a numpy array containing zeros. This dictionary will be used to create a profile matrix of the given DNA seqeunces and eventually a consensus string. 

The `re`, `numpy` and `BioPython` modules are used in this solution. [BioPython](https://docs.python.org/3/library/re.html) is used to parse the FASTA file. Please visit the documentation for more details.

In [1]:
# Import necessary packages
import re
import numpy as np
from Bio import SeqIO

In [2]:
sequences = []
filename = 'input_files/rosalind_cons.txt'

# BioPython is used to parse the FASTA file and add the sequences to a list
with open(filename,'r') as f:
    records = SeqIO.parse(f, 'fasta')
    for record in records:
        seq = str(record.seq)
        sequences.append(seq)

length = len(sequences[0])

# This dictionary will hold the frequency count for each nucleotide at each index 
# The value will be a numpy array of length n, where n is the length of the seqeunces read
# Each index in the numpy array corresponds to the index in the sequence
profile_dict = {'A':np.zeros(length, dtype = int),
                'C': np.zeros(length, dtype = int),
                'G': np.zeros(length, dtype = int),
                'T': np.zeros(length, dtype = int)}

# Loop through each sequence and use the keys from the profile_dict to locate each nucleotide
# with regular expressions
for seq in sequences:
    for key, val in profile_dict.items():
        matches = re.finditer(key, seq)
        # Get the index that each nucleotide appears at and increment the corresponding value 
        # in the profile_dict
        for match in matches:
            val[match.start()]+=1

In [3]:
# To form the consensus string we will use the profile dictionary we created 
# We can loop through the columns of the profile matrix and keep track of the highest count
# We can append the nucleotide with the highest count to our consensus string
rows = len(profile_dict)
cols = len(list(profile_dict.values())[0])

# Loop though the dictionary by column
consensus_string = ''
for column in range(0, cols):
    largest_count = 0
    most_freq_nuc = None
    # Loop though each row in the matrix
    for row in range(0, rows):
        if list(profile_dict.values())[row][column] > largest_count:
            largest_count = list(profile_dict.values())[row][column]
            most_freq_nuc = list(profile_dict.keys())[row]
    consensus_string += most_freq_nuc

In [4]:
with open('output_files/consensus_and_profile.txt', 'a') as f:
    f.write(f'{consensus_string}\n')
    for key, val in profile_dict.items():
        f.write(f'{key}: ')
        for i in val:
            f.write(f'{i} ')
        f.write('\n')

In [5]:
with open('output_files/profile_dict.txt', 'r') as f:
    print(f.read())

AACACAGCAAACCTAAGGGTATTGACTCGCTAACAGAACATATACCAGAGATAAAAGGCGCGCCGCACGTTATAGGAGAATCACAGCAGCGAGACGGGTTCTACCAGGGCATACAGGAACGACTAACAAGAGGCGAAAGCCGTAAGTGTACGTCCCGAGGGGGCGCAACAGCCCGATGTGCTATTTAAAAACCTCGCCCGCCAACGCCCTTGATCAGCGGGGTTGACTAGGAAGAATCTACTGGAACGTCCAAGTGCAAGGCGGATATCGGGTGACAACCAATCCAAAGAGTACAGAACTCTAAAGCACCAAAACTAGACACAACGAAGAACCACGCAGTGAAAACGAGACCACGATGGAACGCGCCACCCCGGTAACACCCGTAGTATAGCGAACGTGACCTCGCGAAAGCCGGAGCCCAAAAAAAAAAGCCAATACACACCTCAGCATTCCTTAACACGTATATGCGACGGAATACCATGGACTCTGCGAGCACCGTATTACTAAGGTGCTAGACTTATACGGGCGCCGATCGACACCCCACGCAAGGAAGGCCTGAAGCAGCGCAGACCCAAGCAAACATACGCACAAGGTCCGTGGACACAGTACAAGCGCCCTCAAGGACAAAATAGAACGATAAAACTAAAAAGAGTCCTCGCGATGGCAGGCCAAGGTCCAAACACAGATATCGATCACGCCCGATAATGCCGGACGGGCATGGCCAAACTCGAGACACAGGTCGGATAACTAACCTCCACAATACGCGCCCACATATCGCACCCAGTGTGATGATCTTCCGACTCATCTGTGCATTCTTCACTAAGCCAAACGGATACCGCATGACAGAAAAATAAGAATGTAAACATAACTGGCGCGTCACAGAACGAGGCACCTGCCGCTTACCCAAAAA
A: 3 4 2 4 1 3 1 2 5 5 4 1 3 3 3 5 2 3 3 1 4 3 1 2 4 1 2 2 2 2 2 4 3 2 3 2 5 3 1 5 0 4 2 